# 编程作业 3 - 多类分类

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [2]:
data = loadmat('data/ex3data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [3]:
data['X'], data['y']

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8))

In [4]:
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

In [5]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

代价函数：
$$J\left( \theta  \right)=\frac{1}{m}\sum\limits_{i=1}^{m}{[-{{y}^{(i)}}\log \left( {{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)-\left( 1-{{y}^{(i)}} \right)\log \left( 1-{{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)]}+\frac{\lambda }{2m}\sum\limits_{j=1}^{n}{\theta _{j}^{2}}$$

Tip：可以通过np.matrix()函数将一个变量转换为numpy型矩阵

In [6]:
def cost(theta, X, y, l):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    cross_cost = -((y.T * np.log(sigmoid(X * theta.T))) + (1 - y).T * np.log(1 - sigmoid(X * theta.T)))
    reg = (l / (2 * len(X))) * np.sum(np.power(theta[:, 1:theta.shape[1]], 2))
    
    whole_cost = np.sum(cross_cost) / len(X) + reg
    
    return whole_cost

如果我们要使用梯度下降法令这个代价函数最小化，因为我们未对${{\theta }_{0}}$ 进行正则化，所以梯度下降算法将分两种情形：
\begin{align}
  & Repeat\text{ }until\text{ }convergence\text{ }\!\!\{\!\!\text{ } \\ 
 & \text{     }{{\theta }_{0}}:={{\theta }_{0}}-a\frac{1}{m}\sum\limits_{i=1}^{m}{[{{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}}]x_{_{0}}^{(i)}} \\ 
 & \text{     }{{\theta }_{j}}:={{\theta }_{j}}-a\frac{1}{m}\sum\limits_{i=1}^{m}{[{{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}}]x_{j}^{(i)}}+\frac{\lambda }{m}{{\theta }_{j}} \\ 
 & \text{          }\!\!\}\!\!\text{ } \\ 
 & Repeat \\ 
\end{align}


In [7]:
def gradient(theta, X, y, learningRate):
    
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    
    err = sigmoid(X * theta.T) - y
    
    grad = ((X.T * err) / len(X)).T + ((learningRate / len(X)) * theta)
    
    grad[0, 0] = np.sum(np.multiply(err, X[:, 0])) / len(X)
    
    return np.array(grad).ravel()

## 构建分类器

In [8]:
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, learning_rate):
    rows = X.shape[0]
    paras = X.shape[1]
    
    all_theta = np.zeros((num_labels, params+1))
    
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))
        
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1, :] = fmin.x
    
    return all_theta

In [10]:
rows = data['X'].shape[0]
params = data['X'].shape[1]

all_theta = np.zeros((10, params + 1))

X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)

theta = np.zeros(params + 1)

y_0 = np.array([1 if label == 0 else 0 for label in data['y']])
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, all_theta.shape

((5000, 401), (5000, 1), (401,), (10, 401))

In [11]:
np.unique(data['y'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [12]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta

array([[-2.38196495e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.30433864e-03, -7.25192320e-10,  0.00000000e+00],
       [-3.18295929e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.45865591e-03, -5.08334500e-04,  0.00000000e+00],
       [-4.79728374e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.87514391e-05, -2.47938954e-07,  0.00000000e+00],
       ...,
       [-7.98780185e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -8.94224211e-05,  7.20239872e-06,  0.00000000e+00],
       [-4.56975018e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.33769778e-03,  1.00024653e-04,  0.00000000e+00],
       [-5.40398177e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.16875200e-04,  7.90582234e-06,  0.00000000e+00]])

In [13]:
all_theta

array([[-2.38196495e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.30433864e-03, -7.25192320e-10,  0.00000000e+00],
       [-3.18295929e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.45865591e-03, -5.08334500e-04,  0.00000000e+00],
       [-4.79728374e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.87514391e-05, -2.47938954e-07,  0.00000000e+00],
       ...,
       [-7.98780185e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -8.94224211e-05,  7.20239872e-06,  0.00000000e+00],
       [-4.56975018e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.33769778e-03,  1.00024653e-04,  0.00000000e+00],
       [-5.40398177e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.16875200e-04,  7.90582234e-06,  0.00000000e+00]])

In [14]:
def predict_all(X, all_theta):
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]
    
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    h = sigmoid(X * all_theta.T)
    
    h_argmax = np.argmax(h, axis=1)
    
    h_argmax = h_argmax + 1
    
    return h_argmax

In [15]:
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print(accuracy)

0.9446
